In [110]:
# Import packages
import pandas as pd
import numpy as np
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

In [88]:
pd.set_option('display.max_columns', None)

In [111]:
# Read attorney dashboard data
data = pd.read_excel('Attorney dashboard data.xlsx')

In [112]:
# Inspect data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84158 entries, 0 to 84157
Data columns (total 28 columns):
MGA                     57046 non-null object
Cases                   84158 non-null object
COAT                    84158 non-null int64
Group                   84158 non-null object
Case Open               84158 non-null datetime64[ns]
LEA                     84158 non-null object
PrecinctNumber          26122 non-null object
Offense Level           84158 non-null object
Team                    31116 non-null object
Adult Disposition       34504 non-null object
Juvenile Disposition    15283 non-null object
Reason for Dismissal    11535 non-null object
EJJCert                 413 non-null object
SpecialtyCourt          1987 non-null object
Stay                    2355 non-null object
SchoolCrime             84158 non-null int64
City                    84152 non-null object
State                   84158 non-null object
Commissioner            74032 non-null float64
Community     

In [113]:
data.shape

(84158, 28)

In [114]:
# Inspect whether the Cases column is unique ID. Based on comparison with dataframe shape, we can find Cases is not unique ID.
# After inspection of data, we can find that rows sharing same cases ID is the case that has different values in OffTypeAll column.
data['Cases'].nunique()

78037

In [115]:
# Inspect whether city column only contain Hennepin county data
# Since there are some format differences for the same city, we need to remove tailing white spaces and cases
a = data['City'].str.rstrip().str.title().sort_values()

In [116]:
# The City column contains cities that don't belong to HC
a.nunique()

225

In [117]:
# List HC cities, there are 45 cities in HC according to the county official site
HC_cities = ['Bloomington', 'Brooklyn Center', 'Brooklyn Park', 'Champlin', 'Chanhassen', 'Corcoran', 'Crystal', 'Dayton', 'Deephaven', 'Eden Prairie', 'Edina', 'Excelsior', 'Golden Valley', 'Greenfield', 'Greenwood', 'Hanover', 'Hopkins', 'Independence', 'Long Lake', 'Loretto', 'Maple Grove', 'Maple Plain', 'Medicine Lake', 'Medina', 'Minneapolis', 'Minnetonka', 'Minnetonka Beach', 'Minnetrista', 'Mound', 'New Hope', 'Orono', 'Osseo', 'Plymouth', 'Richfield', 'Robbinsdale', 'Rockford', 'Rogers', 'St. Anthony', 'St. Bonifacius', 'St. Louis Park', 'Shorewood', 'Spring Park', 'Tonka Bay', 'Wayzata', 'Woodland']

In [118]:
# Create County-City relationship file
cities = pd.DataFrame({'CityId': range(45), 'County': 'Hennepin', 'City': HC_cities})
cities.to_csv('HC_city_list.csv', index = False)

In [119]:
# Create new city column with aligned format, and filter rows belongs to HC
data['City_new'] = data['City'].str.rstrip().str.title()
HC_data = data.loc[data['City_new'].isin(HC_cities)]

In [120]:
# There are 80297 rows left after filtering
HC_data.shape

(80297, 29)

In [102]:
# Inspect Time Span of HC Justice data
print(HC_data['Case Open'].max())
print(HC_data['Case Open'].min())

2019-12-31 16:17:41.707000
2015-01-02 00:00:00


In [121]:
# In order to do yearly and monthly analysis, we need to extract Year and YearMonth from Case Open column
HC_data['Year'] = HC_data['Case Open'].dt.year
HC_data['YearMonth'] = HC_data['Case Open'].dt.strftime('%Y-%m')

In [122]:
# Filter, extract and store yearly data
HC_data.to_csv('HC_Justice_2015-2019.csv', index = None)

In [132]:
# Function to transform raw dashboard data at once

def Justice_tranform(filename):
    data = pd.read_excel(filename)
    data['City_new'] = data['City'].str.rstrip().str.title()
    HC_data = data.loc[data['City_new'].isin(HC_cities)]
    HC_data['Year'] = HC_data['Case Open'].dt.year
    HC_data['YearMonth'] = HC_data['Case Open'].dt.strftime('%Y-%m')
    Yr_max = str(HC_data['Case Open'].max().year)
    Yr_min = str(HC_data['Case Open'].min().year)
    HC_data.to_csv('HC_Justice_' + Yr_min + '-' + Yr_max + '.csv', index = None)
    
    return None

In [133]:
Justice_tranform('Attorney dashboard data.xlsx')